# **INTRODUCTION**
In this notebook, we shall be using the BERT transformer to create model to perform  sentiment analysis on our data.First we shall install the dependancies needed in the notebook.

In [ ]:
!pip install  tensorflow

In [ ]:
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu117

Looking in indexes: https://download.pytorch.org/whl/cu117


In [ ]:
!pip install -q transformers

In [ ]:
!apt-get install git-lfs


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 8 not upgraded.


In [ ]:
!pip install datasets

We shall then make the relevant imports.

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import nltk
import string
import sklearn
import tensorflow as tf
import  torch
from transformers import BertTokenizerFast
from transformers import TFBertModel
from transformers import RobertaTokenizerFast
from transformers import TFRobertaModel
#keras
import tensorflow as tf
from tensorflow import keras
#metrics
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split
import torch
torch.cuda.is_available()
import torch.nn as nn
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, TensorDataset
# Hugging Face Dataset
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, EarlyStoppingCallback, TextClassificationPipeline

from transformers import DataCollatorWithPadding


Lets read our dataset which contains various sentiments from Reddit.The dataset we are using has already gone some cleaning.</br>
This cleaning includes : </br>


1.   Removal of punctuations.
2.   Lowering the case of the words.
3.   Removal of emoji's.
4.   Removal of stop words.
5.   Lemmatization.


In [ ]:
df=pd.read_csv('/content/preprocessed_data_1.csv')
df.head()

,Unnamed: 0,original_text,text,listed_emotions,emotion_count,encoded_labels
0,0,That game hurt.,g a m e h u r t,sadness,1,1
1,1,"You do right, if you don't care then fuck 'em!",r i g h t d o n t c a r e f u c k e m,neutral,1,3
2,2,Man I love reddit.,m a n l o v e r e d d i t,love,1,2
3,3,"[NAME] was nowhere near them, he was by the Fa...",n a m e n o w h e r e n e a r f a l c o n,neutral,1,3
4,4,Right? Considering it’s such an important docu...,r i g h t c o n s i d e r i n g i t ’ s ...,gratitude,1,2


From the above dataset, we can see , the original text, which is the text before cleaning, the text that is cleaned (text column).The listed emotions their count and their encoded labels. </br>
We have four types of emotions and their encoded lables, these are as shown below : </br>

1. Negative emotions : label 1.
2. Positive  emotions :label 2.
3. Neutral emotions : label 3.
4. Ambigous emotions : label 4.

In [ ]:
df=df.rename(columns={"encoded_labels": "labels"})
df.head()

,Unnamed: 0,original_text,text,listed_emotions,emotion_count,labels
0,0,That game hurt.,g a m e h u r t,sadness,1,1
1,1,"You do right, if you don't care then fuck 'em!",r i g h t d o n t c a r e f u c k e m,neutral,1,3
2,2,Man I love reddit.,m a n l o v e r e d d i t,love,1,2
3,3,"[NAME] was nowhere near them, he was by the Fa...",n a m e n o w h e r e n e a r f a l c o n,neutral,1,3
4,4,Right? Considering it’s such an important docu...,r i g h t c o n s i d e r i n g i t ’ s ...,gratitude,1,2


We shall only collect the columns that we need as we go on, the text and the labels

In [ ]:
##Getting the columns we need from the data.
df=df[['text','labels']]
df.head()

,text,labels
0,g a m e h u r t,1
1,r i g h t d o n t c a r e f u c k e m,3
2,m a n l o v e r e d d i t,2
3,n a m e n o w h e r e n e a r f a l c o n,3
4,r i g h t c o n s i d e r i n g i t ’ s ...,2


## 2. CREATING A SIMPLE PIPELINE MODEL.
Below we shall create a simple model that performs sentiment analysis using pipelines that are already pretrained

In [ ]:
#start by getting the specific column that we want to analyze.
data=df.text[2]

In [ ]:
data

'm a n   l o v e   r e d d i t'

In [ ]:
# creating a simple sentiment analysis with pipelines.rs
from transformers import pipeline
sentiment_pipeline = pipeline("sentiment-analysis")
## lets see the sentiments for row 1

sentiment_pipeline(data)


No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'
Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


[{'label': 'POSITIVE', 'score': 0.6460239291191101}]

We can see that our model roughly performs well, hence we should seek to explore other models for this tasks

In [ ]:
# Drop rows with missing values in the "text" column
df.dropna(subset=["text"], inplace=True)

Our dataset has some class imbalance in the labels column,this could negatively affect our model, hence we should remove the class imbalance

In [ ]:
import pandas as pd
from imblearn.over_sampling import SMOTE


In [ ]:
smote = SMOTE(sampling_strategy='auto', random_state=42)


In [ ]:
# We shall use the randomuderSampler because it shall resample our data and do away with class imbalance
## The sampler shall also help us reduce the number of rows we have which is good because it
## saves us time when running our model

# Import the necessary libraries
from imblearn.under_sampling import RandomUnderSampler

# Create the RandomUnderSampler object
under_sampler = RandomUnderSampler(random_state=42)

# Perform random undersampling on the DataFrame
data_resampled, _ = under_sampler.fit_resample(df, df['labels'])

# Now data_resampled will contain the randomly undersampled data


In [ ]:
data_resampled

,text,labels
0,w e l l i ’ d s a y p r e t t y g o o ...,0
1,h a p p y n a m e s a d h e s i m a g ...,0
2,n a r e d i t i n g p o s t s o r r y ...,0
3,p i t y d e c e n t l u n c h e s n e v ...,0
4,i ’ m s o r r y g a m e r e a l l y i ...,0
...,...,...
42739,n a m e d a m n c u r t i s,3
42740,d o n a t e d p e n n y s o m e o n e e ...,3
42741,y o u ’ v e g o t s l i g h t s h i t e ...,3
42742,c a p n a m e w i n s c h a m p s y e ...,3


In [ ]:
data_resampled.labels.value_counts()

0    10686
1    10686
2    10686
3    10686
Name: labels, dtype: int64

From the above, we can see that our number of rows have reduced and that we have a balnce in the labels columns.

In [ ]:
# Step 2: Split the data into training and testing sets
train_data, test_data = train_test_split(data_resampled, test_size=0.2, random_state=42)


In [ ]:
print("Legnth of our training data set",len(train_data))
print("Legnth of our testing data set",len(test_data))

Legnth of our training data set 34195
Legnth of our testing data set 8549


# 3. Tokenization.
If we want to use a bert model, we sahll use a bert tokenizer to tokenize the text column so that we can feed it into our model.

In [ ]:
# instantiate the pretrained tokenizer
# Tokenizer from a pretrained model
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

# Take a look at the tokenizer
tokenizer


BertTokenizerFast(name_or_path='bert-base-cased', vocab_size=28996, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True)

We then convert our dataframe to a huggingframe dataset so that we can be able to easily work with the data

In [ ]:
# Convert pyhton dataframe to Hugging Face arrow dataset
hg_train_data = Dataset.from_pandas(train_data)
hg_test_data = Dataset.from_pandas(test_data)


Even though our data is reduced, the 40000 rows are still alot to work with and can increase our runtime legnth. Therefore, we shall reduce the legnth of our data to 10000 rows for the train and 2500 for the test randomly .

In [ ]:
hg_train_data = hg_train_data.shuffle(seed=42).select([i for i in list(range(10000))])
hg_test_data = hg_test_data.shuffle(seed=42).select([i for i in list(range(2500))])


In [ ]:
# Funtion to tokenize data
def tokenize_dataset(data):
    return tokenizer(data["text"],
                     max_length=32,
                     truncation=True,
                     padding="max_length")

# Tokenize the dataset
dataset_train = hg_train_data.map(tokenize_dataset)
dataset_test = hg_test_data.map(tokenize_dataset)

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2500 [00:00<?, ? examples/s]

In [ ]:
# Take a look at the data
print(dataset_train)
print(dataset_test)


Dataset({
    features: ['text', 'labels', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 10000
})
Dataset({
    features: ['text', 'labels', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 2500
})


In [ ]:
# Check the first record
dataset_train[0]

{'text': 'h a h a   m a y b e   5 5   t i t l e   p a r t y   m a y',
 'labels': 2,
 '__index_level_0__': 24253,
 'input_ids': [101,
  177,
  170,
  177,
  170,
  182,
  170,
  194,
  171,
  174,
  126,
  126,
  189,
  178,
  189,
  181,
  174,
  185,
  170,
  187,
  189,
  194,
  182,
  170,
  194,
  102,
  0,
  0,
  0,
  0,
  0,
  0],
 'token_type_ids': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0]}

We are using the bert model, hence we shall instantiate a bert model.

In [ ]:
# Load model
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=4)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
!pip install accelerate -U
! pip install -U transformers


In [ ]:
from torch.optim import AdamW
from transformers import TrainingArguments
from transformers import Trainer

since our model is pretrained, we shall create our training arguments and pass them into the trainer then train our model

In [ ]:
# Set up training arguments
training_args = TrainingArguments(
    output_dir="Chepchumba/autotrain-data-sentiment_analysis",
    logging_dir='Chepchumba/autotrain-data-sentiment_analysis/logs',
    logging_strategy='epoch',
    logging_steps=100,
    num_train_epochs=2,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    learning_rate=5e-6,
    seed=42,
    save_strategy='epoch',
    save_steps=100,
    evaluation_strategy='epoch',
    eval_steps=100,
    load_best_model_at_end=True,

)
optimizer = AdamW(model.parameters(), lr=5e-6)



In [ ]:
!pip install evaluate

We shall use  a data collator that helps us to orgarnize and structure our data in a format that is suitable for training the machine model.

In [ ]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

We shall also create a function called compute_metric that will enable us to evaluate the model by checking its accuracy.

In [ ]:
# Model performance evaluation
import evaluate

In [ ]:
import numpy as np
from datasets import load_metric

import numpy as np
from datasets import load_metric

# Function to compute the metric
def compute_metrics(eval_pred):
    metric = evaluate.load("accuracy")
    logits, labels = eval_pred
    probabilities = tf.nn.softmax(logits)
    predictions = np.argmax(logits, axis=1)
    return metric.compute(predictions=predictions, references=labels)


In [ ]:
# Train the model
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_train,
    eval_dataset=dataset_test,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=1)]
)





Now that we have the model, lets go ahead and use it to train on our data.

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,1.385700,1.362610,0.318400
2,1.347200,1.327487,0.343600


TrainOutput(global_step=5000, training_loss=1.366470263671875, metrics={'train_runtime': 11332.9282, 'train_samples_per_second': 1.765, 'train_steps_per_second': 0.441, 'total_flos': 328894725120000.0, 'train_loss': 1.366470263671875, 'epoch': 2.0})

We can see that our model has a very low accuracy and a high training loss, lets go ahead and Fine tune our model so tha we can find the best parameters available.

## 3) HYPERPARAMETER TUNING.
Hyperparameter tuning is basically trying to find the best parameter for your model in a wide range of parameters.We shall use the Optuna library which has a wide range of hyperparameters  in python, to find the best hyperparameters for our model

In [ ]:
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 390.6/390.6 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 7.9 MB/s eta 0:00:00


In [ ]:
import optuna
from transformers import Trainer, TrainingArguments


In [ ]:
def objective(trial: optuna.Trial):
    # Define hyperparameter search space
    learning_rate = trial.suggest_loguniform("learning_rate", 1e-6, 1e-4)
    per_device_train_batch_size = trial.suggest_categorical("per_device_train_batch_size", [4, 8, 16])
    num_train_epochs = trial.suggest_int("num_train_epochs", 1, 5)

    # Set up training arguments
    training_args = TrainingArguments(

        output_dir="Chepchumba/autotrain-data-sentiment_analysis",
        logging_dir='Chepchumba/autotrain-data-sentiment_analysis/logs',
        logging_strategy='epoch',
        logging_steps=100,
        num_train_epochs=num_train_epochs,
        per_device_train_batch_size=per_device_train_batch_size,
        per_device_eval_batch_size=4,
        learning_rate=learning_rate,
        seed=42,
        save_strategy='epoch',
        save_steps=100,
        evaluation_strategy='epoch',
        eval_steps=100,
        load_best_model_at_end=True
    )

    # Create and train the model
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=dataset_train,
        eval_dataset=dataset_test,
        compute_metrics=compute_metrics,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=1)]
    )

    trainer.train()

    # Get the evaluation metric (accuracy) from the trained model
    accuracy = trainer.evaluate().metrics["eval_accuracy"]

    return accuracy


In [ ]:
# Create an Optuna study
study = optuna.create_study(direction="maximize")

# Optimize the objective function
study.optimize(objective, n_trials=10)

# Get the best hyperparameters
best_params = study.best_params
print("Best Hyperparameters:", best_params)


[I 2023-07-21 13:30:50,743] A new study created in memory with name: no-name-91c31693-6ce7-4e57-809c-9cfb7f334a17
<ipython-input-69-8776dbf5043e>:3: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform("learning_rate", 1e-6, 1e-4)
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss,Accuracy
1,1.405700,1.404926,0.240800


In [ ]:
# Predictions
y_test_predict = trainer.predict(dataset_test)

# Take a look at the predictions
y_test_predict

In [ ]:
# Predicted logits
y_test_logits = y_test_predict.predictions

# First 5 predicted probabilities
y_test_logits[:5]
